In [ ]:
import pandas as pd
import numpy as np
import os

In [ ]:
df1 = pd.read_csv("/Users/SeanOMalley1/Downloads/Final_CajaDelAmor_2018_PPI.csv")

# isolate parents dataset
df1_1 = df1.iloc[:,:6]

# clean parents dataset
df1_1 = df1_1[df1_1['Nº FAM'].isnull() == False]
df1_1.rename(columns={'Nº FAM':'FAM_N'}, inplace=True)
df1_1.FAM_N = df1_1.FAM_N.astype('int')

# father present
df1_1['PADRE'] = np.where(df1_1['APELLIDOS  PADRE'].isnull() == True, 0, 1)

# mother present
df1_1['MADRE'] = np.where(df1_1['APELLIDOS MADRE'].isnull() == True, 0, 1)

# reduce it to just what we want
df_padres = df1_1.iloc[:,[0,-3,-2,-1]]

# isolate rest of household dataset
df1_2 = df1.iloc[:,6:-1]

# clean parents dataset
df1_2 = df1_2[df1_2['Nº FAM.1'].isnull() == False]
df1_2.rename(columns={'Nº FAM.1':'FAM_N'}, inplace=True)
df1_2.FAM_N = df1_2.FAM_N.astype('int')
df1_2.EDAD = np.where(df1_2.EDAD.str.contains('MESES')|df1_2.EDAD.str.contains('MES') == True, '0.5', df1_2.EDAD)
df1_2.EDAD = df1_2.EDAD.astype('float64')

# count females
df1_2['F'] = np.where(df1_2['SEXO'] == 'F', 1, 0)

# count males
df1_2['M'] = np.where(df1_2['SEXO'] == 'M', 1, 0)

# count children
df1_2['NINOS'] = np.where(df1_2['EDAD'] <= 18, 1, 0)

# could old
df1_2['MAYORES'] = np.where(df1_2['EDAD'] >= 65, 1, 0)

# reduce it to just what we want
df_familia = df1_2.groupby(['FAM_N'])['F','M','NINOS','MAYORES'].sum().reset_index()

# combine both dataframes

df_todo = df_padres.merge(df_familia, left_on='FAM_N', right_on='FAM_N', how='outer')

# add father and mother to F & M totals

df_todo['F'] = df_todo['F'] + df_todo['MADRE']
df_todo['M'] = df_todo['M'] + df_todo['PADRE']

df_todo.to_csv('/Users/SeanOMalley1/AnacondaProjects/SeanOMalleyCodePortfolio/Development Economics/PovertySurveys/CDA_PPI_CLEAN_2018.csv')
